# Rock Paper Scissors with Jetbot - Data Collection

## 1. Initialization
* widgets are used to create an interactable or displayable interface in the notebook.
* os is use to interact with I?O of the file directories.
* traitlets is to link and transform the data type
* display is to stream the live video feed
* Camera is to interact with the camera onboard the jetbot
* uuid to create unique identifiers when saving images

In [ ]:
import ipywidgets.widgets as widgets
import os
from traitlets import dlink
from IPython.display import display
from jetbot import Camera, bgr8_to_jpeg
from uuid import uuid1

## 2. Display live camera feed
Initialized to collect ``224x224`` ``RGB`` images.

In [ ]:
camera = Camera.instance(width=224, height=224)

image = widgets.Image(format='jpeg', width=224, height=224)

camera_link = dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

display(image)

## 3. Initialize Database Directories
Create a folder ``dataset`` that will contain three sub-folders ``rock``, ``paper``, and ``scissors``.

In [ ]:
rock_dir = 'dataset/rock'
paper_dir = 'dataset/paper'
scissor_dir = 'dataset/scissor'

# we have this "try/except" statement because these next functions can throw an error if the directories exist already
try:
    os.makedirs(rock_dir)
    os.makedirs(paper_dir)
    os.makedirs(scissor_dir)
except FileExistsError:
    print('Directories not created becasue they already exist')

## 4. Create Interactive Inferface
Count the number of images in each dataset and create buttons to easily add to the dataset.

In [ ]:
button_layout = widgets.Layout(width='128px', height='64px')
rock_button = widgets.Button(description='Add Rock', button_style='success', layout=button_layout)
paper_button = widgets.Button(description='Add Paper', button_style='success', layout=button_layout)
scissor_button = widgets.Button(description='Add Scissor', button_style='success', layout=button_layout)
free_count = widgets.IntText(layout=button_layout, value=len(os.listdir(rock_dir)))
paper_count = widgets.IntText(layout=button_layout, value=len(os.listdir(paper_dir)))
scissor_count = widgets.IntText(layout=button_layout, value=len(os.listdir(scissor_dir)))

display(widgets.HBox([rock_count, rock_button]))
display(widgets.HBox([paper_count, paper_button]))
display(widgets.HBox([scissor_count, scissor_button]))

Create callbacks functions to save the images to the desired directory.

Use ``uuid1`` method to generate unique identifiers.

In [ ]:
def save_snapshot(directory):
    image_path = os.path.join(directory, str(uuid1()) + '.jpg')
    with open(image_path, 'wb') as f:
        f.write(image.value)

def save_rock():
    global rock_dir, rock_count
    save_snapshot(free_dir)
    rock_count.value = len(os.listdir(rock_dir))
    
def save_paper():
    global paper_dir, paper_count
    save_snapshot(paper_dir)
    paper_count.value = len(os.listdir(paper_dir))
    
def save_scissor():
    global scissor_dir, scissor_count
    save_snapshot(scissor_dir)
    scissor_count.value = len(os.listdir(scissor_dir))

Add ``on_click`` events to each button to save the ``Image`` widget output in the JPEG format.

Use a 'lambda' function to ignore the return parameter of ``on_click`` as it is unneeded.

In [ ]:
rock_button.on_click(lambda x: save_rock())
paper_button.on_click(lambda x: save_paper())
scissor_button.on_click(lambda x: save_scissor())

## 5. Collect Data
1. Different orientations
2. Different lighting
3. Varied object / collision types; walls, ledges, objects
4. Different textured floors / objects;  patterned, smooth, glass, etc.

In [ ]:
display(image)
display(widgets.HBox([rock_count, rock_button]))
display(widgets.HBox([paper_count, paper_button]))
display(widgets.HBox([scissor_count, scissor_button]))

## 6. Zip Data (Optional: only if using Cloud GPU outside of Jetson Nano)
> The ! prefix indicates that we want to run the cell as a *shell* (or *terminal*) command.
> The -r flag in the zip command below indicates *recursive* so that we include all nested files, the -q flag indicates *quiet* so that the zip command doesn't print any output

In [ ]:
!zip -r -q dataset.zip dataset